In [1]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import csv
import numpy as np
import cv2
from tqdm import tqdm

from edge_detection import edgeDetection
from interest_point_detection import display_interest_points_matrix
from detect_laser import *
from torchinfo import summary

import matplotlib.pyplot as plt

In [2]:
# Prepare data 

data_path = Path("./data")
csv_list = list(data_path.glob("*.csv"))
file_map = dict()
test_map = dict()
i = 0
l = 0
for csv_file in csv_list:   
    with open(csv_file, newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        header = next(reader)

        # First 2 of every file will be for testing
        for k in range(2):
            row = next(reader)
            if len(row) > 0:
                test_map[l] = dict()
                test_map[l]["name"] = row[0]
                
                for j in range(1, 3):
                    test_map[l][header[j]] = row[j]

                l+=1
        
        for row in reader:
        
            if len(row) > 0:
                file_map[i] = dict()
                file_map[i]["name"] = row[0]
                
                for j in range(1, 3):
                    file_map[i][header[j]] = row[j]

                i+=1

In [3]:
data_map = dict()
test_data_map = dict()
index = 0
index_test = 0

In [4]:
# Getting images with laser inside them for training
for i in tqdm(file_map):
    #img = cv2.imread(file_map[i]["name"])

    val = 20
    count=0

    for k in range(5,val):
        for j in range(5,val):
            laser_np = (int(file_map[i]["laser.x"]), int(file_map[i]["laser.y"]))

            #temp = img[laser_np[1]-val+k:laser_np[1]+k,laser_np[0]-val+j:laser_np[0]+j]
            # exit(1)
            count+=1

            # Add to data_map
            # 0 is no laser, 1 is yes laser
            data_map[index] = dict()
            data_map[index]["file_name"] = file_map[i]["name"]
            data_map[index]["image"] = (laser_np[1]-val+k, laser_np[0]-val+j)
            data_map[index]["label"] = 1
            index += 1

            # cv2.namedWindow("burger")
            # cv2.imshow("burger", temp)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
print(count)

100%|██████████████████████████████████████████████████████████████████████████████████| 326/326 [00:00<00:00, 1223.04it/s]

225


In [5]:
# Getting images without any laser for training
for i in tqdm(file_map):
  #  img = cv2.imread(file_map[i]["name"])
    val=20
    laser_np = (int(file_map[i]["laser.x"]), int(file_map[i]["laser.y"]))
    left = laser_np[0] - 20*2 + 5
    right = laser_np[0] + 35
    top = laser_np[1]-20*2+5
    bottom = laser_np[1]+35

    count = 0
    
    for k in range(50,3000-val-50,85):
        for j in range(50,4000-val-50,85):

            if j >= left and j < right and k >= top and k < bottom:
                continue
            # cv2.rectangle(img, (j,k), (j+val, k+val), color=(0,0,255), thickness=2)

            data_map[index] = dict()
            data_map[index]["file_name"] = file_map[i]["name"]
            data_map[index]["image"] = (k, j)
            data_map[index]["label"] = 0
            index += 1
            count+=1
            # temp = img[k:k+val, j:j+val]
print(count)

100%|███████████████████████████████████████████████████████████████████████████████████| 326/326 [00:01<00:00, 233.24it/s]

1564


In [6]:
# Getting images with laser inside them for training
for i in tqdm(test_map):
    #img = cv2.imread(file_map[i]["name"])

    val = 20
    count=0

    for k in range(5,val):
        for j in range(5,val):
            laser_np = (int(test_map[i]["laser.x"]), int(test_map[i]["laser.y"]))

            #temp = img[laser_np[1]-val+k:laser_np[1]+k,laser_np[0]-val+j:laser_np[0]+j]
            # exit(1)
            count+=1

            # Add to data_map
            # 0 is no laser, 1 is yes laser
            test_data_map[index] = dict()
            test_data_map[index]["file_name"] = test_map[i]["name"]
            test_data_map[index]["image"] = (laser_np[1]-val+k, laser_np[0]-val+j)
            test_data_map[index]["label"] = 1
            index_test += 1

            # cv2.namedWindow("burger")
            # cv2.imshow("burger", temp)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

100%|████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 1327.84it/s]


In [7]:
# Getting images without any laser for testing
for i in tqdm(test_map):
  #  img = cv2.imread(test_map[i]["name"])
    val=20
    laser_np = (int(test_map[i]["laser.x"]), int(test_map[i]["laser.y"]))
    left = laser_np[0] - 20*2 + 5
    right = laser_np[0] + 35
    top = laser_np[1]-20*2+5
    bottom = laser_np[1]+35
    
    for k in range(50,3000-val-50,50):
        for j in range(50,4000-val-50,50):

            if j >= left and j < right and k >= top and k < bottom:
                continue
            # cv2.rectangle(img, (j,k), (j+val, k+val), color=(0,0,255), thickness=2)

            test_data_map[index_test] = dict()
            test_data_map[index_test]["file_name"] = test_map[i]["name"]
            test_data_map[index_test]["image"] = (k, j)
            test_data_map[index_test]["label"] = 0
            index_test += 1
            # temp = img[k:k+val, j:j+val]

100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 64.25it/s]


In [8]:
class LaserDetectionDataset(Dataset):
    """Laser Detection dataset."""

    def __init__(self, data_map):
        """
        Arguments:
            data_map (string): map to training data.
        """
        self.data_map = data_map

    def __len__(self):

        return len(self.data_map)

    def __getitem__(self, idx):
        print(idx)
        file_name = self.data_map[idx]["file_name"]
        k,j = self.data_map[idx]["image"]
        label = self.data_map[idx]["label"]

        img = cv2.imread(file_name)
        img_clone = img.copy()
        print(img_clone.shape)
        output_img = img_clone[k:k+20, j:j+20]
        # cv2.namedWindow("burger")
        # cv2.imshow("burger", output_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        one_hot = [0,0]
        one_hot[label] = 1
        return output_img, np.array(one_hot)

In [ ]:
laser_dataset = LaserDetectionDataset(data_map = data_map)
laser_dataloader = DataLoader(laser_dataset, shuffle=False)

for i, (img, label) in tqdm(enumerate(laser_dataloader)):

    display = np.reshape(img.numpy().astype(np.uint8), (20,20,3))
    print(display.shape)
    cv2.namedWindow("burger")
    cv2.imshow("burger", display)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if i == 20:
        cv2.destroyAllWindows()
        break

0it [00:00, ?it/s]

0
(3000, 4000, 3)
(20, 20, 3)


1it [00:01,  1.78s/it]

1
(3000, 4000, 3)
(20, 20, 3)


2it [00:02,  1.18s/it]

2
(3000, 4000, 3)
(20, 20, 3)


In [2]:
print(len(data_map))

NameError: name 'data_map' is not defined